<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

In [11]:
import pandas as pd
import folium
from folium import Map, Marker, PolyLine, features, RegularPolygonMarker, DivIcon
from folium.plugins import PolyLineTextPath
import math
import networkx as nx
import time
import random
import multiprocessing

In [ ]:
# 2.3.1

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep = "\t")
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep = "\t")

charging_station_nodes = {323346405, 87120378, 2374444198, 1345740157, 2351742223}

coords = {
    row['osmid']: (row['y'], row['x'])  # Folium notar (lat, lon)
    for _, row in nodes.iterrows()
}

# búum til graf með stefnu og þyngd fyrir edges
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['length'])

## Need to reverse in some cases because it's directed
G_rev = G.reverse()

In [13]:
# 2.3.2

def shortest_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.dijkstra_path_length(graph, source=node_id, target=charger_id, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

In [14]:
# 2.3.3

def process_node_wrapper(args):
    row, charger_nodes, G, charger_only = args
    node_id = row['osmid']
    if node_id in charger_nodes:
        color = "orange"
        popup_text = f"🔌 Hleðslustöð <br>Node ID: {node_id}"
    elif charger_only == False:
        color = "red" if row['primary'] else "blue"
        distance, closest = shortest_distance_to_charger(node_id, G, charger_nodes)
        popup_text = (f"🚗 Node ID: {node_id}<br>Primary: {row['primary']}<br>"
                      f"Fjarlægð frá næstu hleðslustöð: {distance:.2f} meters<br>"
                      f"Næsta hleðslustöð: {closest}")
    else:
        return None
    
    return {
        'location': [row['y'], row['x']],
        'color': color,
        'popup_text': popup_text
    }

# keyrsla gæti tekið sirka 5-10 mín
def create_map(charger_nodes, map_name, chargers_only = False):
    center_lat = nodes['y'].mean()
    center_lon = nodes['x'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    start_time = time.time()

    row_args = [
        (row, charger_nodes, G, chargers_only)
        for _, row in nodes.iterrows()
    ]

    # ryzen 7 go brrrr
    with multiprocessing.Pool() as pool:
        marker_data_list = pool.map(process_node_wrapper, row_args)

    for md in marker_data_list:
        if md is None:
            continue
        
        folium.CircleMarker(
            location=md['location'],
            radius=4,
            color=md['color'],
            fill=True,
            fill_color=md['color'],
            fill_opacity=0.8,
            popup=folium.Popup(md['popup_text'], max_width=250)
        ).add_to(m)

    print(f"Keyrsla tók {int(time.time() - start_time)}s")

    for _, row in edges.iterrows():
        if row['u'] in coords and row['v'] in coords:
            start = coords[row['u']]
            end = coords[row['v']]
            PolyLine(
                locations=[start, end],
                color='gray',
                weight=1,
                opacity=0.5,
                popup=row.get('name', '')
            ).add_to(m)

            # Reiknum gráðu á ör-iconinu og setjum svo á endann á línunni
            # vantar að laga betur, gerir kortið mjög hægt líka.
            """angle = calculate_angle(start, end)
            folium.map.Marker(
                location=end,
                icon=DivIcon(
                    icon_size=(150, 36),
                    icon_anchor=(7, 20),
                    html=f'<div style="transform: rotate({angle}deg); color: gray; font-size: 16px;">&#8594;</div>'
                )
            ).add_to(m)"""

    m.save(map_name)
    print(f"Kort geymt i skra: {map_name}")

create_map(charging_station_nodes, "kort_2_3_3.html")

Keyrsla tók 25s
Kort geymt i skra: kort_2_3_3.html


In [15]:
# 2.3.4

## it's in the code above

In [16]:
# 2.3.5

## Heuristic for A*
## Needlessly complicated on such a small scale
## Calculate the distance between two points on the surface of a sphere
## It's somewhat off up here since the Earth isn't a perfect sphere

def calc_spherical_distance(node_id, target_id):
    lat_source,lon_source = coords[node_id]
    lat_target,lon_target = coords[target_id]

    radius = 6373.0

    delta_lat = math.radians(lat_target) - math.radians(lat_source)
    delta_lon = math.radians(lon_target) - math.radians(lon_source)

    a = math.sin(delta_lat / 2)**2 + math.cos(lat_source) * math.cos(lat_target) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000 #metres

    return distance

def astar_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.astar_path_length(graph, source=node_id, target=charger_id, heuristic=calc_spherical_distance, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

## Return the sum for later use
def process_astar_row(args):
    row, charging_station_nodes, G = args
    node_id = row['osmid']

    distance, closest = astar_distance_to_charger(node_id, G, charging_station_nodes)
    return distance if distance != float("inf") else 0.0

def run_astar(charging_station_nodes, benchmark):
    distance_sum = 0.0
    if benchmark:
        start_time = time.time()

    row_args = [(row, charging_station_nodes, G) for _, row in nodes.iterrows()]
    
    with multiprocessing.Pool() as pool:
        distances = pool.map(process_astar_row, row_args)

    distance_sum = sum(distances)

    if benchmark:
        print(f"Keyrsla tók {int(time.time() - start_time)}s")

    return distance_sum

run_astar(charging_station_nodes, True)

Keyrsla tók 19s


52496231.653249905

In [ ]:
# 2.3.6

## For all primary nodes, run Dijkstra on the reversed graph
def simple_dijkstra(charger_id):
    distances = nx.single_source_dijkstra_path_length(G_rev, charger_id, weight='weight')
    distance_sum = sum(distances.values())

    return distance_sum

def multi_dijkstra(candidate_charger_id: int, placed_charger_ids: set):
    chargers = placed_charger_ids.copy()
    chargers.add(candidate_charger_id)

    distances = nx.multi_source_dijkstra_path_length(G_rev, chargers, weight="weight")
    distance_sum = sum(distances.values())

    return distance_sum

def place_optimal_charger(primary_nodes, used_nodes):
    best_distance = float("inf")
    best_node = None

    for node in primary_nodes:
        distance = multi_dijkstra(node.osmid, used_nodes)

        if distance < best_distance and distance != 0:
            best_distance = distance
            best_node = node

    return best_node, best_distance

primary_nodes = []
for _, row in nodes.iterrows():
    if row['primary'] == True:
        primary_nodes.append(row)

optimal_node, distance = place_optimal_charger(primary_nodes, set())
print(f"Best node for k = 1 was {optimal_node} with a distance of {distance}")

Best node for k = 1 was osmid       34827739
x         -21.845736
y          64.114075
primary         True
Name: 589, dtype: object with a distance of 71813532.58901912


In [30]:
# 2.3.7

## Always add the new best node. Obviously have to remove the previously best node

def greedy_k_chargers(max_iterations):
    used_nodes = set()
    best_nodes = []

    start_time = time.time()
    for k in range(max_iterations):
        best_node, dist = place_optimal_charger(primary_nodes, used_nodes)
        print(f"Added new node {best_node.osmid}, current distance is {dist}")
        best_nodes.append(best_node)
        used_nodes.add(best_node.osmid)
        # Python is awful btw
        primary_nodes[:] = [node for node in primary_nodes if not node.equals(best_node)]

    print(f"Keyrsla tók {int(time.time() - start_time)}s")
    return best_nodes

greedy_nodes = greedy_k_chargers(10)
create_map([node.osmid for node in greedy_nodes], "kort_2_3_7.html", True)

Added new node 34827739, current distance is 71813532.58901912
Added new node 4159611763, current distance is 55332780.57688582
Added new node 470316424, current distance is 44243359.814382106
Added new node 1204996745, current distance is 37461685.95094731
Added new node 470320635, current distance is 34361904.73860957
Added new node 253702373, current distance is 31990647.604471616
Added new node 2948755314, current distance is 29861914.732461683
Added new node 62975525, current distance is 27813118.38432762
Added new node 252165232, current distance is 26337079.36916443
Added new node 251765347, current distance is 24909868.79067309
Keyrsla tók 204s
Keyrsla tók 1s
Kort geymt i skra: kort_2_3_7.html


In [ ]:
# 2.3.8

## Pick random starting node
### Run greedy K for 5 nodes
#### Pick best node, then choose the furthest of the remaining 4
##### Recurse
###### Pick solution with smallest sum of distances at the end

def 